### install evaluate

In [1]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Evaluate models on the Hub

[Using the evaluator](https://huggingface.co/docs/evaluate/en/base_evaluator)

In [4]:
from evaluate import evaluator
from evaluate import combine
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, pipeline



data = load_dataset("imdb", split="test").shuffle(seed=42).select(range(1000))
task_evaluator = evaluator("text-classification")

# 1. pass model or name path
eval_results = task_evaluator.compute(
    model_or_pipeline="lvwerra/distilbert-imdb",
    data=data,
    metric=combine(["accuracy", "recall", "precision", "f1"]),  #calculate multiple metrices. if not specified By default the "accuracy" metric is computed. or single metric specify : metric="accuracy" etc
    input_column="text",                                                  #input_column="text": with this argument the column with the data for the pipeline can be specified. see imdb dataset
    label_column="label",                                                 #label_column="label": with this argument the column with the labels for the evaluation can be specified. see imdb dataset. this is the postive negative label in 0/1
    label_mapping={"NEGATIVE": 0, "POSITIVE": 1},                         #label_mapping=None: the label mapping aligns the labels in the pipeline output with the labels need for evaluation. E.g. the labels in label_column can be integers (0/1) whereas the pipeline can produce label names such as "positive"/"negative". With that dictionary the pipeline outputs are mapped to the labels.
)

# 2. Pass an instantiated model is also possible
# model = AutoModelForSequenceClassification.from_pretrained("lvwerra/distilbert-imdb")
# eval_results = task_evaluator.compute(
#     model_or_pipeline=model,
#     data=data,
#     label_mapping={"NEGATIVE": 0, "POSITIVE": 1}
# )

# 3. Pass an instantiated pipeline is also possible
# pipe = pipeline("text-classification", model="lvwerra/distilbert-imdb")
# eval_results = task_evaluator.compute(
#     model_or_pipeline=pipe,
#     data=data,
#     label_mapping={"NEGATIVE": 0, "POSITIVE": 1}
# )


print(eval_results)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

{'accuracy': 0.918, 'recall': 0.9180327868852459, 'precision': 0.9142857142857143, 'f1': 0.9161554192229039, 'total_time_in_seconds': 10.937744419000069, 'samples_per_second': 91.42652833091343, 'latency_in_seconds': 0.010937744419000068}


## Evaluate models using Transformer


https://huggingface.co/docs/evaluate/en/transformers_integrations

The metrics in evaluate can be easily integrated with the Trainer. The Trainer accepts a compute_metrics keyword argument that passes a function to compute metrics. One can specify the evaluation interval with evaluation_strategy in the TrainerArguments, and based on that, the model is evaluated accordingly, and the predictions and labels passed to compute_metrics.

[Customized Evaluation Metrics with Hugging Face Trainer](https://medium.com/@rakeshrajpurohit/customized-evaluation-metrics-with-hugging-face-trainer-3ff00d936f99)

In [11]:
! pip install datasets transformers torch evaluate accelerate sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Prepare and tokenize dataset
dataset = load_dataset("yelp_review_full")
print(dataset)

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))


### evaluate using huggingface evaluate

In [10]:
from datasets import load_dataset
import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import numpy as np

#metric = evaluate.combine(["precision", "recall", "accuracy", "f1"])
metric1 = evaluate.load("precision")
metric2 = evaluate.load("recall")
metric3 = evaluate.load("accuracy")
metric4 = evaluate.load("f1")
# Setup evaluation
def compute_metrics(eval_pred):



    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision = metric1.compute(predictions=predictions, references=labels,  average="micro")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels,  average="micro")["recall"]
    accuracy = metric3.compute(predictions=predictions, references=labels,)["accuracy"]
    f1 = metric4.compute(predictions=predictions, references=labels,  average="micro")["f1"]
    return {"precision": precision, "recall": recall, "accuracy": accuracy, "f1": f1}
    #return metric.compute(predictions=predictions, references=labels)

# Load pretrained model and evaluate model after each epoch
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
#output issue: You  can see it poduces first metric (Precision) for all other metrices if we try to compute multiple
# Epoch	Training Loss	Validation Loss	Precision	Recall	Accuracy	F1
# 1	No log	1.119715	0.527000	0.527000	0.527000	0.527000
# 2	No log	1.013838	0.566000	0.566000	0.566000	0.566000
# 3	No log	1.034243	0.592000	0.592000	0.592000	0.592000


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,Accuracy,F1
1,No log,1.119715,0.527000,0.527000,0.527000,0.527000
2,No log,1.013838,0.566000,0.566000,0.566000,0.566000
3,No log,1.034243,0.592000,0.592000,0.592000,0.592000


TrainOutput(global_step=375, training_loss=1.03717578125, metrics={'train_runtime': 397.1044, 'train_samples_per_second': 7.555, 'train_steps_per_second': 0.944, 'total_flos': 789354427392000.0, 'train_loss': 1.03717578125, 'epoch': 3.0})

### evaluate using sklearn

will fix the above issue of computing multiple metric in Trainer using HuggingFac evaluate

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch" #if max_steps=max_steps not provided it defaults to 3 steps/epoch training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.128622,0.481000,0.533712,0.481000,0.440829
2,No log,0.979463,0.572000,0.582543,0.572000,0.571721
3,No log,1.039855,0.596000,0.602211,0.596000,0.597519


TrainOutput(global_step=375, training_loss=0.9462509765625, metrics={'train_runtime': 392.8908, 'train_samples_per_second': 7.636, 'train_steps_per_second': 0.954, 'total_flos': 789354427392000.0, 'train_loss': 0.9462509765625, 'epoch': 3.0})